In [1]:
import torch
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
from collections import Counter
from tqdm import tqdm
import os
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Downloading datasets

In [26]:
import urllib.request
import os

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'

folder_path = 'data'
if not os.path.exists(folder_path):
    os.makedirs(folder_path) 

filename = os.path.join(folder_path, 'input.txt')
urllib.request.urlretrieve(url, filename)

('data\\input.txt', <http.client.HTTPMessage at 0x2317d91d460>)

In [64]:
import datasets
dataset = datasets.load_dataset("FredZhang7/stable-diffusion-prompts-2.47M")
train = dataset["train"]
with open('data/all_promts.txt', 'w') as f:
    for line in train['text']:
        f.write(line + '\n')

### Data preparation

In [31]:
with open(r'c:\Users\vladk\Desktop\Machine Learning\Learning\DiveIntoDL\11_Transformers\building_GPT\1\data\voina-i-mir.txt', 'r', encoding='utf-8') as f:
    text = f.read()

all_chars = sorted(list(set(text)))
print(f"len(all_chars) = {len(all_chars)}")

chars_counter = Counter(text)
sorted_chars_counter = dict(sorted(chars_counter.items(), key=lambda item: item[1], reverse=True))
deleted_rare_chars = {v[0] : v[1] for v in sorted_chars_counter.items() if v[1] > 100}

print("---* text is filtered *---")
text = ''.join([t for t in text if t in deleted_rare_chars])
chars = sorted(list(deleted_rare_chars.keys()))
vocab_size = len(chars)
print(f"len(chars) = {len(chars)} | vocab_size = {vocab_size}")

len(all_chars) = 180
---* text is filtered *---
len(chars) = 130 | vocab_size = 130


In [32]:
"""
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
"""
from tiktoken._educational import *
enc = tiktoken.encoding_for_model("gpt-2")
vocab_size = enc.n_vocab

data = torch.tensor(enc.encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data, val_data = data[:n], data[n:]

In [33]:
block_size = 256
batch_size = 32
n_embd = 384
n_layer, n_head = 6, 6
dropout = 0.2

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')

### Model

In [34]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        B, T, C = x.shape
        k, q = self.key(x), self.query(x) #(B, T, head_size)
        wei = q @ k.transpose(-2, -1) * C**-0.5#(B, T, head_size) @ (B, head_size, T) ---> (B, T, T)
        wei = wei.masked_fill(self.tril[:T][:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [35]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # Shape (B (batch_size), T (block_size), C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb #(B,T,C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

m = BigramLanguageModel()
m = m.to(device)

In [36]:
eval_iters = 200
eval_interval = 1000
max_iters = 20000
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

@torch.no_grad
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

### Training

In [37]:
for iter in tqdm(range(max_iters), desc=f"Training Progress"):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

Training Progress:   0%|          | 1/20000 [00:13<75:20:04, 13.56s/it]

step 0: train loss 11.0671, val loss 11.0777


Training Progress:   5%|▌         | 1001/20000 [02:17<21:57:46,  4.16s/it]

step 1000: train loss 1.5832, val loss 1.8460


Training Progress:  10%|█         | 2001/20000 [04:22<21:12:14,  4.24s/it]

...

step 7000: train loss 1.0619, val loss 1.4984


Training Progress:  40%|████      | 8001/20000 [16:55<12:19:02,  3.70s/it]

step 8000: train loss 1.0393, val loss 1.5086


Training Progress:  41%|████▏     | 8251/20000 [17:23<24:46,  7.90it/s]   


KeyboardInterrupt: 

In [39]:
context = torch.zeros((1, block_size), dtype=torch.long, device=device)
print(enc.decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!226!, слов... — по
адрезал, поддевался головой.
Графина пошла вмешаться над танцами рассказали: прощальные плечами,
лошал, как сам сестра. Граф видимо родные глаза заметились сабли, салфите.
Не было, нынче еще ребеном к продолжающему беспр�ойству.
Но в деле графа А с толкой опящего обходят власти на мрах. Выражающие
ребенчики вопросительских ена новых гостей, необычайное радостное самовара,
и сильное местное радостно оживленным, не столько того, но и это дом
писителют армии?
Письмо кроме вел этот эти общества.
«Есть я награждаю силону?» думал он, но продолжая
его останавливаться и своим тоном страх подчинение говорить с тем, что сомнение
свобода было на ее военности; но в этом становитости по станцованическому воинстве
действительност

In [18]:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

49.386577 M parameters


In [25]:
PATH = 'trained_models/gpt2-tokenizer-40000_loss-2.18.pth'
os.makedirs(os.path.dirname(PATH), exist_ok=True)
torch.save(m.state_dict(), PATH)

In [1]:
total_params = sum(p.numel() for p in m.parameters())

NameError: name 'm' is not defined

### Testing